In [1]:
%pip install -q "qiskit>=1.0" "qiskit-aer>=0.13" "qiskit-ibm-runtime>=0.17" "pennylane-qiskit>=0.42"
# Qiskit Aer & PennyLane noise model w/ IBM Simulator


Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Qiskit noise models
# Note: was unable to create IBM account to import real backend, so we'll use this
from qiskit_ibm_runtime.fake_provider import FakeSherbrooke, FakeJakarta
from qiskit_aer.noise import NoiseModel

# Convert (optional, for PennyLane 'pl_noise' path)
from pennylane_qiskit import load_noise_model

from utils import qalton

# some helper functions

def bins_from_counts(counts: dict, n_bins: int) -> np.ndarray:
    """Aggregate shot counts (little-endian bitstrings) into probabilities for each n-bins."""
    bins = np.zeros(n_bins, dtype=float)
    total = 0
    for bitstr, c in counts.items():
        total += c
        bin_idx = bitstr[::-1].find("1")  # wire-0 is rightmost char
        if 0 <= bin_idx < n_bins:
            bins[bin_idx] += c
    if total > 0:
        bins /= total
    return bins

def bins_from_probs(probs: np.ndarray, n_bins: int) -> np.ndarray:
    """Aggregate an exact probability vector (2^n_bins long) into n_bins by position."""
    bins = np.zeros(n_bins, dtype=float)
    for idx, p in enumerate(probs):
        bitstr = format(idx, f"0{n_bins}b")[::-1]  # little-endian
        if bitstr.count("1") == 1:
            bin_idx = bitstr.find("1")
            bins[bin_idx] += p
    return bins

#what is tvd?
def tvd(p, q) -> float:
    """Total Variation Distance between two distributions p and q."""
    p = np.asarray(p, dtype=float)
    q = np.asarray(q, dtype=float)
    return 0.5 * np.abs(p - q).sum()


# Noise Models

In [3]:
# Again, fake backends so not real-time, but it'll have "realistic" calibrations
backend_A = FakeSherbrooke()   # large device, modern calibration
backend_B = FakeJakarta()      # smaller device, different noise profile

noise_A_qiskit = NoiseModel.from_backend(backend_A)
noise_B_qiskit = NoiseModel.from_backend(backend_B)

noise_A_pl = load_noise_model(noise_A_qiskit)
noise_B_pl = load_noise_model(noise_B_qiskit)


C:\Users\13174\AppData\Local\Temp\ipykernel_24720\3437833596.py:3: DeprecationWarning: V1 fake backends are deprecated as of qiskit-ibm-runtime 0.24 and will be removed no sooner than 3 months after the release date. Please use V2 fake backends instead.
  backend_B = FakeJakarta()      # smaller device, different noise profile
C:\Users\13174\AppData\Local\Temp\ipykernel_24720\3437833596.py:6: DeprecationWarning:  from_backend using V1 based backend is deprecated as of Aer 0.15 and will be removed no sooner than 3 months from that release date. Please use backends based on V2.
  noise_B_qiskit = NoiseModel.from_backend(backend_B)
c:\Users\13174\anaconda3\Lib\site-packages\pennylane_qiskit\noise_models.py:135: UserWarning: Readout errors are not supported currently and will be skipped.
  warn("Readout errors are not supported currently and will be skipped.")


# Experiment w/ Params

In [4]:
levels       = 12          # number of bins
shots_noisy  = 50_000      # sampling shots for noisy runs
bias_biased  = 0.20        # how skewed we want it to be, if <0.5, then left skewed, else right skewed
bias_walk    = 0.50        # Hadamard coin

# choose which path to test: "qiskit" (Aer device) or "pl" (default.mixed + add_noise)
noise_path = "qiskit"     

# Runner (One Config)

In [5]:
def run_case(noise_label, noise_obj, distribution):
    """
    Run one distribution under one noise model and compare to ideal.
    noise_label: str label for reporting
    noise_obj:   the noise model object (Qiskit NoiseModel or PL NoiseModel)
    distribution: "biased" or "walk"
    """
    is_qiskit = (noise_path == "qiskit")
    # Set distribution parameters
    if distribution == "biased":
        bias = bias_biased
        coherence = False
    elif distribution == "walk":
        bias = bias_walk
        coherence = True
    else:
        raise ValueError("distribution must be 'biased' or 'walk'")

    # --- ideal (noiseless) reference: probs on a small statevector ---
    qc_ideal = qalton.build_galton_circuit(
        levels=levels, num_shots=1,       # probs, analytic
        bias=bias, coherence=coherence,
        return_counts=False               # return probabilities
    )
    probs_ideal = qc_ideal()
    bins_ideal  = bins_from_probs(probs_ideal, n_bins=levels)

    # --- noisy run: counts ---
    if is_qiskit:
        qc_noisy = qalton.build_galton_circuit(
            levels=levels, num_shots=shots_noisy,
            bias=bias, coherence=coherence,
            qiskit_noise_model=noise_obj,   # Aer path
            return_counts=True
        )
    else:
        qc_noisy = qalton.build_galton_circuit(
            levels=levels, num_shots=shots_noisy,
            bias=bias, coherence=coherence,
            pl_noise=noise_obj,             # PL path
            return_counts=True
        )

    counts_noisy = qc_noisy()
    bins_noisy   = bins_from_counts(counts_noisy, n_bins=levels)
    metric_tvd   = tvd(bins_noisy, bins_ideal)

    return bins_ideal, bins_noisy, metric_tvd


# Distributions on Both Noise Models

In [7]:
results = []

# Model A
if noise_path == "qiskit":
    noise_A = noise_A_qiskit
    noise_B = noise_B_qiskit
else:
    noise_A = noise_A_pl
    noise_B = noise_B_pl

for label, nm in [("FakeSherbrooke", noise_A), ("FakeJakarta", noise_B)]:
    for dist in ["biased", "walk"]:
        ideal, noisy, d = run_case(label, nm, dist)
        results.append({
            "noise_model": label,
            "path": noise_path,
            "distribution": dist,
            "TVD": d,
            "ideal": ideal,
            "noisy": noisy,
        })

df = pd.DataFrame(results)[["noise_model", "path", "distribution", "TVD"]]
display(df)


WireError: Did not find some of the wires Wires([0, 24]) on device with wires Wires([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]).